# Artificial Neural Network

***

**Like in Python, we will not have to use our own resource to train the ANN. Here we will use h2o package which connect to a server and train the ANN on that. More importantly h2o have parameter tunning feature which automatically tune the parameter as per requirement.**

***

### Data preprocessing

In [1]:
# Importing the dataset
dataset = read.csv('Churn_Modelling.csv')

In [2]:
head(dataset, 5)

RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
dataset = dataset[4:14]
dataset = dataset[-c(3)]

In [4]:
head(dataset, 5)

CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
619,France,42,2,0.00,1,1,1,101348.88,1
608,Spain,41,1,83807.86,1,0,1,112542.58,0
502,France,42,8,159660.80,3,1,0,113931.57,1
699,France,39,1,0.00,2,0,0,93826.63,0
850,Spain,43,2,125510.82,1,1,1,79084.10,0


In [5]:
# Encoding the categorical variables as factors
dataset$Geography = as.numeric(factor(dataset$Geography,
                                      levels = c('France', 'Spain', 'Germany'),
                                      labels = c(1, 2, 3)))

In [6]:
# Splitting the dataset into the Training set and Test set
# install.packages('caTools')
library(caTools)
set.seed(1234)
split = sample.split(dataset$Exited, SplitRatio = 0.8)
training_set = subset(dataset, split == TRUE)
test_set = subset(dataset, split == FALSE)

In [7]:
# Feature Scaling
training_set[-10] = scale(training_set[-10])
test_set[-10] = scale(test_set[-10])

In [8]:
head(training_set, 10)

CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
-0.32830179,-0.8987119,0.291777057,-1.0542790,-1.2236044,-0.9108785,0.6454683,0.9568669,0.02811437,1
-0.44280238,0.3057939,0.196731800,-1.3997035,0.1171130,-0.9108785,-1.5490691,0.9568669,0.22279194,0
-1.54617163,-0.8987119,0.291777057,1.0182685,1.3305715,2.5418712,0.6454683,-1.0449467,0.24694885,1
0.50442971,-0.8987119,0.006641287,-1.3997035,-1.2236044,0.8154963,-1.5490691,-1.0449467,-0.10271041,0
2.07621044,0.3057939,0.386822313,-1.0542790,0.7842567,-0.9108785,0.6454683,0.9568669,-0.35910821,0
-0.05766405,0.3057939,0.481867570,1.0182685,0.5962053,0.8154963,0.6454683,-1.0449467,0.87000926,1
1.78475441,-0.8987119,1.052139108,0.6728439,-1.2236044,0.8154963,0.6454683,0.9568669,-1.55950657,0
-2.85772375,1.5102997,-0.943811277,-0.3634298,0.6168575,4.2682460,0.6454683,-1.0449467,0.34113035,1
-1.55658077,-0.8987119,0.481867570,-0.3634298,1.0488596,0.8154963,-1.5490691,0.9568669,-0.43117249,0
0.34829256,-0.8987119,-1.133901790,-1.0542790,0.9297231,-0.9108785,0.6454683,0.9568669,-0.48708284,0


In [9]:
head(test_set, 10)

,CreditScore,Geography,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
11,-1.23748505,-0.9177619,-0.7617858,0.3979153,0.4121241,0.7772138,-1.5418136,-0.975061,-0.3708209,0
13,-1.76284916,-0.9177619,-0.4720783,1.7886199,-1.2343242,0.7772138,0.6482625,-0.975061,-1.3077491,0
17,0.02540944,1.4879050,1.8455814,-1.3404654,0.9057542,-0.9142199,0.6482625,-0.975061,-1.6754875,1
21,0.82355875,-0.9177619,0.2039058,1.0932676,-1.2343242,0.7772138,0.6482625,1.025064,1.2052900,0
25,1.97531853,-0.9177619,-0.0858017,0.0502392,-1.2343242,-0.9142199,0.6482625,1.025064,1.4959939,0
30,-2.41955429,-0.9177619,-0.9549241,-1.6881415,-0.2708713,0.7772138,0.6482625,1.025064,-0.8347297,0
37,-1.62140497,0.2850715,-0.7617858,-0.6451131,1.1100313,-0.9142199,-1.5418136,1.025064,0.2179836,0
46,1.26809761,1.4879050,-0.6652166,-0.2974369,0.5316239,0.7772138,0.6482625,1.025064,0.4343295,0
51,0.48005145,1.4879050,0.4936132,1.7886199,0.6436647,0.7772138,0.6482625,-0.975061,1.6774538,0
55,-0.49995467,1.4879050,0.3004749,-1.3404654,0.3552986,-0.9142199,0.6482625,-0.975061,-1.0687604,1


### Fiting ANN to the Training set

In [10]:
# install.packages('h2o')

In [11]:
library(h2o)
h2o.init(nthreads = -1)


----------------------------------------------------------------------

Your next step is to start H2O:
    > h2o.init()

For H2O package documentation, ask for help:
    > ??h2o

After starting H2O, you can use the Web UI at http://localhost:54321
For more information visit http://docs.h2o.ai

----------------------------------------------------------------------


Attaching package: ‘h2o’

The following objects are masked from ‘package:stats’:

    cor, sd, var

The following objects are masked from ‘package:base’:

    &&, %*%, %in%, ||, apply, as.factor, as.numeric, colnames,
    colnames<-, ifelse, is.character, is.factor, is.numeric, log,
    log10, log1p, log2, round, signif, trunc




H2O is not running yet, starting it now...

Note:  In case of errors look at the following log files:
    /tmp/Rtmp8aN7Ch/h2o_baka_started_from_r.out
    /tmp/Rtmp8aN7Ch/h2o_baka_started_from_r.err


Starting H2O JVM and connecting: ....... Connection successful!

R is connected to the H2O cluster: 
    H2O cluster uptime:         5 seconds 838 milliseconds 
    H2O cluster version:        3.16.0.2 
    H2O cluster version age:    27 days  
    H2O cluster name:           H2O_started_from_R_baka_fmq912 
    H2O cluster total nodes:    1 
    H2O cluster total memory:   0.82 GB 
    H2O cluster total cores:    4 
    H2O cluster allowed cores:  4 
    H2O cluster healthy:        TRUE 
    H2O Connection ip:          localhost 
    H2O Connection port:        54321 
    H2O Connection proxy:       NA 
    H2O Internal Security:      FALSE 
    H2O API Extensions:         XGBoost, Algos, AutoML, Core V3, Core V4 
    R Version:                  R version 3.4.2 (2017-09-28) 



In [12]:
classifier = h2o.deeplearning(y = 'Exited', 
                              training_frame = as.h2o(training_set),
                              activation = 'Rectifier',
                              hidden = c(6, 6),
                              epochs = 100,
                              train_samples_per_iteration = -2)

  |======================================================================| 100%
  |======================================================================| 100%


### Predicting the Test set results

In [13]:
prob_pred = h2o.predict(classifier, 
                        newdata = as.h2o(test_set[-10]))
y_pred = (prob_pred > 0.5)

# Convert h2o object to vector
y_pred = as.vector(y_pred)

  |======================================================================| 100%
  |======================================================================| 100%


In [14]:
head(y_pred, 20)

[1] 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0

In [15]:
head(test_set[, 10], 20)

[1] 0 0 1 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0 0

### Making the Confusion Matrix

In [16]:
cm = table(test_set[, 10], y_pred)
cm

   y_pred
       0    1
  0 1530   63
  1  218  189

### Calculating Accuracy

In [17]:
(cm[1, 1] + cm[2, 2]) / 2000

[1] 0.8595

### Disconnect from server

In [18]:
h2o.shutdown()

Are you sure you want to shutdown the H2O instance running at http://localhost:54321/ (Y/N)? y


[1] TRUE